# NHANES Analysis Using 2013-2014, 2015-2016, & 2017-2018 Data

### Importing Packages

In [ ]:
import os
os.environ['QT_API'] = 'PyQt6'

import pandas as pd
import numpy as np
import plotnine as pn
from janitor import clean_names
from matplotlib import rcParams
import seaborn as sns
import matplotlib.pyplot as plt
from pyhere import here
import pyarrow as pa
import pyarrow.parquet as pq

# Set some options
pd.set_option('display.max_columns', None)
pd.set_option('mode.copy_on_write', True)
rcParams.update({'savefig.bbox': 'tight'}) # Keeps plotnine legend from being cut off


### Blood Analyses to Create Latent Profiles

In [ ]:
hdl_table = pq.read_table(here('data/cleaned/hdl.parquet'))
hdl = hdl_table.to_pandas()

ldl_table = pq.read_table(here('data/cleaned/ldl.parquet'))
ldl = ldl_table.to_pandas()

bio_table = pq.read_table(here('data/cleaned/bio.parquet'))
bio = bio_table.to_pandas()

# total_chol_table = pq.read_table(here('data/cleaned/total_chol.parquet'))
# total_chol = total_chol_table.to_pandas()

print(hdl.head(), '\n')
print(ldl.head(), '\n')
print(bio.head(), '\n')
# print(total_chol.head(), '\n')

In [ ]:
ldl_sub = ldl.dropna(subset = ['trigly_mg_dl', 'ldl_mg_dl'], how = 'all')
bio_sub = bio.dropna(subset = ['albumin_g_dl', 'alp_iu_l', 'ast_u_l', 'alt_u_l', 'ggt_u_l', 'total_bilirubin_mg_dl', 'tri_mg_dl'], how = 'all')

In [ ]:
blood_df = (
  ldl_sub
  .merge(hdl,
         'left',
         'id')
  .merge(bio_sub,
         'left',
         'id')
)

In [ ]:
blood_train = blood_df.sample(frac = .8, random_state = 100825)

miss_pct = blood_train.isnull().sum()/len(blood_train)*100
miss_pct.sort_values(ascending = False)

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')

blood_traini = imp.fit_transform(blood_train)
blood_traini = pd.DataFrame(blood_traini, columns = blood_train.columns)

In [ ]:
pn.ggplot.show(
  pn.ggplot(blood_traini, pn.aes('ast_u_l'))
  + pn.geom_histogram(color = 'black', fill = 'seagreen')
  + pn.theme_light()
)

pn.ggplot.show(
  pn.ggplot(blood_traini, pn.aes('alt_u_l'))
  + pn.geom_histogram(color = 'black', fill = 'seagreen')
  + pn.theme_light()
)

Variables to probably keep are: **alt_bi**, **ast_bi**, **ggt_bi**, **hdl_bi**, **ldl_bi**, **trigly_bi**

Unsure about: albumin_g-dl, alp_iu_l, total_bilirubin_mg_dl

In [ ]:
blood_traini = blood_traini.drop(columns = 'tri_mg_dl')
blood_traini.drop(columns = 'id').corr().round(2)

In [ ]:
from stepmix.stepmix import StepMix

lpa_model = []
for i in range(5):
  mod = StepMix(n_components = i + 1,
          measurement = 'continuous',
          verbose = 1,
          n_init = 20,
          random_state = 100825)
  lpa_model.append(mod)

for mod in lpa_model:
  mod.fit(blood_traini.drop(columns = 'id'))

for i, mod in enumerate(lpa_model, start = 1):
  blood_traini[f'class{i}'] = mod.predict(blood_traini[['trigly_mg_dl', 'ldl_mg_dl',
                                                        'hdl_mg_dl', 'albumin_g_dl',
                                                        'alp_iu_l', 'ast_u_l',
                                                        'alt_u_l', 'ggt_u_l',
                                                        'total_bilirubin_mg_dl']])


In [ ]:
rel_entropy = [lpa_model[i].relative_entropy(blood_traini[['trigly_mg_dl', 'ldl_mg_dl',
                                          'hdl_mg_dl', 'albumin_g_dl',
                                          'alp_iu_l', 'ast_u_l',
                                          'alt_u_l', 'ggt_u_l',
                                          'total_bilirubin_mg_dl']]).round(3) for i in np.arange(1, len(lpa_model))]

sabic_values = [lpa_model[i].sabic(blood_traini[['trigly_mg_dl', 'ldl_mg_dl',
                                          'hdl_mg_dl', 'albumin_g_dl',
                                          'alp_iu_l', 'ast_u_l',
                                          'alt_u_l', 'ggt_u_l',
                                          'total_bilirubin_mg_dl']]).round(2) for i in range(len(lpa_model))]

aic_values = [lpa_model[i].aic(blood_traini[['trigly_mg_dl', 'ldl_mg_dl',
                                          'hdl_mg_dl', 'albumin_g_dl',
                                          'alp_iu_l', 'ast_u_l',
                                          'alt_u_l', 'ggt_u_l',
                                          'total_bilirubin_mg_dl']]).round(2) for i in range(len(lpa_model))]

bic_values = [lpa_model[i].bic(blood_traini[['trigly_mg_dl', 'ldl_mg_dl',
                                          'hdl_mg_dl', 'albumin_g_dl',
                                          'alp_iu_l', 'ast_u_l',
                                          'alt_u_l', 'ggt_u_l',
                                          'total_bilirubin_mg_dl']]).round(2) for i in range(len(lpa_model))]

In [ ]:
rel_entropy.insert(0, 1)

In [ ]:
fit_df = pd.DataFrame({'classes': [1, 2, 3, 4, 5],
              'aic': aic_values,
              'bic': bic_values,
              'sabic': sabic_values,
              'rel_entropy': rel_entropy})

fit_df = fit_df.melt(id_vars = ['classes', 'rel_entropy'], value_vars = ['aic', 'bic', 'sabic'])

In [ ]:
pn.ggplot.show(
  pn.ggplot(fit_df, pn.aes('classes', 'value'))
  + pn.geom_line(pn.aes(color = 'variable'))
  + pn.geom_point(pn.aes(color = 'variable'))
  + pn.geom_text(pn.aes(label = 'rel_entropy'))
  + pn.facet_wrap('variable', scales = 'free')
  + pn.theme_light()
)

In [ ]:
[blood_traini[f'class{i+1}'].value_counts(normalize = True) for i in range(len(lpa_model))]

In [ ]:
(
  [blood_traini.
  groupby('class2')[i]
  .mean()
  .reset_index()
  for i in ['trigly_mg_dl', 'ldl_mg_dl',
  'hdl_mg_dl', 'albumin_g_dl',
  'alp_iu_l', 'ast_u_l',
  'alt_u_l', 'ggt_u_l',
  'total_bilirubin_mg_dl']]
)

In [ ]:
(
  [blood_traini.
  groupby('class3')[i]
  .mean()
  .reset_index()
  for i in ['trigly_mg_dl', 'ldl_mg_dl',
  'hdl_mg_dl', 'albumin_g_dl',
  'alp_iu_l', 'ast_u_l',
  'alt_u_l', 'ggt_u_l',
  'total_bilirubin_mg_dl']]
)

In [ ]:
(
  [blood_traini.
  groupby('class4')[i]
  .mean()
  .reset_index()
  for i in ['trigly_mg_dl', 'ldl_mg_dl',
  'hdl_mg_dl', 'albumin_g_dl',
  'alp_iu_l', 'ast_u_l',
  'alt_u_l', 'ggt_u_l',
  'total_bilirubin_mg_dl']]
)

In [ ]:
(
  [blood_traini.
  groupby('class5')[i]
  .mean()
  .reset_index()
  for i in ['trigly_mg_dl', 'ldl_mg_dl',
  'hdl_mg_dl', 'albumin_g_dl',
  'alp_iu_l', 'ast_u_l',
  'alt_u_l', 'ggt_u_l',
  'total_bilirubin_mg_dl']]
)

In [ ]:
for i in ['trigly_mg_dl', 'ldl_mg_dl', 
          'hdl_mg_dl', 'albumin_g_dl',
          'alp_iu_l', 'ast_u_l',
          'alt_u_l', 'ggt_u_l', 'total_bilirubin_mg_dl']:
  pn.ggplot.show(
    pn.ggplot(
      blood_traini,
      pn.aes(i)
    )
    + pn.geom_histogram(
      pn.aes(fill = 'factor(class2)'),
      bins = 20
    )
    + pn.facet_wrap('class2')
    + pn.theme_light()
  )

# pn.ggplot.show(
#   pn.ggplot(
#     blood_traini,
#     pn.aes('trigly_mg_dl',
#            'ldl_mg_dl')
#   )
#   + pn.geom_point(pn.aes(color = 'factor(class2)'))
#   + pn.theme_light()
# )

# pn.ggplot.show(
#   pn.ggplot(
#     blood_traini,
#     pn.aes('hdl_mg_dl',
#            'ldl_mg_dl')
#   )
#   + pn.geom_point(pn.aes(color = 'factor(class2)'))
#   + pn.theme_light()
# )

# pn.ggplot.show(
#   pn.ggplot(
#     blood_traini,
#     pn.aes('ast_u_l',
#            'alt_u_l')
#   )
#   + pn.geom_point(pn.aes(color = 'factor(class2)'))
#   + pn.theme_light()
# )

In [ ]:
for i in ['trigly_mg_dl', 'ldl_mg_dl', 
          'hdl_mg_dl', 'albumin_g_dl',
          'alp_iu_l', 'ast_u_l',
          'alt_u_l', 'ggt_u_l', 'total_bilirubin_mg_dl']:
  pn.ggplot.show(
    pn.ggplot(
      blood_traini,
      pn.aes(i)
    )
    + pn.geom_histogram(
      pn.aes(fill = 'factor(class3)'),
      position = 'dodge',
      bins = 20
    )
    + pn.facet_wrap('class3')
    + pn.theme_light()
  )

### Merging in the Data From Database 

In [ ]:
data_tbl = pq.read_table(here('data/nhanes_data_2013_2017.parquet'))
data = data_tbl.to_pandas()

In [ ]:
joined = blood_traini.merge(data, 'right', on = 'id')
print(joined.head())
(joined.shape)

In [ ]:
# joined = joined.dropna(subset = ['albumin_g_dl', 'alp_iu_l', 'ast_u_l', 'alt_u_l', 'ggt_u_l', 'total_bilirubin_mg_dl', 'trigly_mg_dl', 'ldl_mg_dl', 'hdl_mg_dl'], how = 'all')
# joined.head()

### Splitting Into Training & Testing

In [ ]:
train = joined.sample(frac = .8, random_state = 100825)
print(train.head(), '\n\n')
print(joined.shape)

In [ ]:
train = train.drop(columns = ['class1', 'class3', 'class4', 'class5', 'birth_country'])
print(train.head())

In [ ]:
miss_pct1 = train.isnull().sum()/len(train)*100
miss_pct1.sort_values(ascending = False)

Conducting Imputation on Training Data

In [ ]:
train.head()

In [ ]:
train['birth_country'].value_counts(normalize = True)

In [ ]:
train['heard_my_plate'] = np.where(train['heard_my_plate'] == 1, 1, 0)

train['race_ethnic'] = train['race_ethnic'].astype('object')
train['length_us'] = train['length_us'].astype('object')
train['ed'] = train['ed'].astype('object')
train['marital'] = train['marital'].astype('object')
train['annual_house_income'] = train['annual_house_income'].astype('object')

# train_num = train.select_dtypes(['int8', 'int32', 'float']).drop(columns = 'id')

In [ ]:
estimator = RandomForestRegressor(
    n_estimators = 100,
    max_depth = 2,
    random_state = 100825,
    n_jobs = -1
)

In [ ]:

numeric_imputer = IterativeImputer(
    estimator = estimator,
    max_iter = 10,
    random_state = 100825
)

train_num_imp = pd.DataFrame(
    numeric_imputer.fit_transform(train_num),
    columns = train_num.columns
)

train_num_imp.head()

In [ ]:
# running multiple immputation iterations

m = 5

imputed_datasets = []

for i in range(m):
    imputer = IterativeImputer(
        estimator=estimator,
        random_state=i,  # Different seed for each imputation
        sample_posterior=True,  # Adds randomness like MICE
        max_iter=10
    )
    imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    imputed_datasets.append(imputed)
    
    

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse_output = False, drop = 'if_binary')

In [ ]:
train_cat = onehot.fit_transform(train[['race_ethnic', 'length_us', 'ed', 'marital', 'annual_house_income']])
column_names = onehot.get_feature_names_out(['race_ethnic', 'length_us', 'ed', 'marital', 'annual_house_income'])
column_names

In [ ]:
# dummy code categorical columns



# train_cat_imp = coder.fit_transform(train[['race_ethnic', 'length_us']])
# train_cat_imp

# column_names = coder.get_feature_names_out(['race_ethnic', 'length_us'])
# column_names

# encode_df = pd.DataFrame(data = train_cat_imp,
#                          columns = column_names)

# encode_df.head()


# Test Data Preprocessing & Predictions

In [ ]:
# test_model = StepMix(n_components = ,
#                      measurement = 'continuous',
#                      verbose = 1,
#                      n_init = 20,
#                      random_state = 100825)
# test_model.fit(blood_testi.drop(columns = 'id'))

# blood_testi[f'lat_class'] = test_model.predict(blood_testi[['trigly_mg_dl', 'ldl_mg_dl',
#                                                             'hdl_mg_dl', 'albumin_g_dl',
#                                                             'alp_iu_l', 'ast_u_l',
#                                                             'alt_u_l', 'ggt_u_l',
#                                                             'total_bilirubin_mg_dl']])